# Resolución utilizando $A^*$

## Imports

In [23]:
from aima_libs.hanoi_states import StatesHanoi, ProblemHanoi
from aima_libs.tree_hanoi import NodeHanoi
from aima_libs.aima import PriorityQueue
import tracemalloc
import numpy

## Estructura general

In [5]:
def generate_new_problem(n_disks: int = 5) -> ProblemHanoi:
    """
    Inicializa un problema de Hanoi para `n_disks` discos y 3 pegs.
    El estado inicial es con el peg izquierdo completo.
    El estado final (goal) es con el peg derecho completo.
    """
    full_peg = list(range(n_disks,0,-1))
    initial_state = StatesHanoi(full_peg, [], [], max_disks=n_disks)
    goal_state = StatesHanoi([], [], full_peg, max_disks=n_disks)

    return ProblemHanoi(initial=initial_state, goal=goal_state)

In [8]:
def a_star_search(n_disks, heur, max_iters = 10000):
    """
    Resuelve un problema de Hanoi para `n_disks` discos y 3 pegs.
    Utiliza para ello una búsqueda A* con heurística `heur` de tipo `StatesHanoi` -> `float`.
    También realiza como máximo `max_iters` iteraciones, tras las cuales retorna no solución.
    En ambos casos se retornan métricas asociadas.
    """
    problem = generate_new_problem(n_disks)

    pq = PriorityQueue('min', lambda ns: ns.path_cost + heur(ns.state))

    pq.append(NodeHanoi(problem.initial))

    seen = set()
    n_iters = 0

    while len(pq)>0:
        if n_iters > max_iters:
            metrics = {'solved': False, 'cost': None}
            break

        _, ns = pq.pop()
        n_iters += 1
        seen.add(ns.state)

        if problem.goal_test(ns.state):
            metrics = {'solved': True, 'cost': ns.state.accumulated_cost}
            break

        for next_ns in ns.expand(problem):
            if next_ns.state not in seen:
                pq.append(next_ns)

    metrics["nodes_explored"] = n_iters
    metrics["states_visited"] = len(seen)
    metrics["nodes_in_frontier"] = len(pq)
    metrics["optimal_cost"] = 2**n_disks - 1

    return (ns if metrics['solved'] else None), metrics

## Heurística Simple

Se utiliza la cantidad de discos en la varilla (peg) izquierda. Como cada disco en la misma requiere al menos 1 movimiento para ser movido a la varilla derecha, esta heúristica es consistente/admisible y por lo tanto el camino generado será óptimo. 

In [9]:
def heur_left_peg_disks(s: StatesHanoi) -> float:
    return len(s.get_state()[0])

In [11]:
# realizamos una corrida
node, metrics = a_star_search(5, heur_left_peg_disks)

In [12]:
# mostramos metricas
metrics

{'solved': True,
 'cost': 31.0,
 'nodes_explored': 302,
 'states_visited': 206,
 'nodes_in_frontier': 33,
 'optimal_cost': 31}

In [14]:
%%timeit
# mini bench de tiempos
node, metrics = a_star_search(5, heur_left_peg_disks)

56.9 ms ± 6.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Heurística Mejorada

Se utiliza la cantidad de discos en las varillas izquierda y medias, bajo la misma lógica antes expresada, pero además se agrega 2 veces la cantidad de discos en la varilla derecha para los cuales hay algún disco más grande en otra varilla.

La lógica del segundo componente es que para agregar dicho disco más grande faltante se requiere mover el disco más chico dos veces, uno para liberar espacio y otro para restaurarlo en la varilla derecha.

In [15]:
def heur_non_right_plus_gap(s: StatesHanoi) -> float:
    right_peg = s.get_state()[2]
    last = s.number_of_disks + 1 # fake base disk
    gap = False  # if missing a disk below
    total = 0
    for disk in right_peg:
        if last > disk + 1:
            gap = True
        if gap:
            total += 2
        last = disk
    return total + (s.number_of_disks - len(right_peg))

In [16]:
# realizamos una corrida
node, metrics = a_star_search(5, heur_non_right_plus_gap)

In [17]:
# mostramos metricas
metrics

{'solved': True,
 'cost': 31.0,
 'nodes_explored': 240,
 'states_visited': 158,
 'nodes_in_frontier': 21,
 'optimal_cost': 31}

In [18]:
%%timeit
# mini bench de tiempos
node, metrics = a_star_search(5, heur_non_right_plus_gap)

43.3 ms ± 2.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
# mini bench de memoria
measurements = []

tracemalloc.start()
for i in range(20):
    node, metrics = a_star_search(5, heur_non_right_plus_gap)
    _, mem_peak = tracemalloc.get_traced_memory()
    tracemalloc.reset_peak()
    measurements.append(mem_peak / 1024**2)
tracemalloc.stop()

# promedio de MB
mu = np.mean(measurements)
s = np.std(measurements)

print(f"Consumo pico de memoria en MB: {mu:.2f}+-{s:.2f}")

Consumo pico de memoria en MB: 0.22+-0.01


## Generación de animación

In [19]:
node.generate_solution_for_simulator(
    initial_state_file="./simulator/initial_state.json",
    sequence_file="./simulator/sequence.json"
)

In [21]:
%%bash

cd ./simulator
uv run ./simulation_hanoi.py

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
